In [1]:
#be sure to start up Stanford Parser server following steps here: https://github.com/nltk/nltk/wiki/Stanford-CoreNLP-API-in-NLTK
from nltk.parse import CoreNLPParser
from nltk.tree import Tree, ParentedTree
from datasets import load_dataset
import time #for tracking time to run
import tracemalloc #for tracking memory usage
_START_RUNTIME = time.time()
tracemalloc.start()
#first, load in the MSRP training data
df = load_dataset('glue', 'mrpc', split='train')
labels = df['label']
sentence1 = df['sentence1']
sentence2 = df['sentence2']
#also load in the MSRP test data, we will preprocess this as well
df = load_dataset('glue', 'mrpc', split='test')
labels_test = df['label']
sentence1_test = df['sentence1']
sentence2_test = df['sentence2']
#initialize the Standford parser
parser = CoreNLPParser(url='http://localhost:9000')

Found cached dataset glue (C:/Users/rbrow/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (C:/Users/rbrow/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [2]:
#implementation of SPO algorithm as outlined in the paper's pseudocode (Algorithm 1)
def spo(sentence):
    tree = parser.raw_parse(sentence)
    tree = next(tree) #need to pull the Tree out of the iter
    
    subject = ""
    predicate = ""
    obj = ""
    #make empty lists instead of strings
    #subject = []
    #predicate = []
    #obj = []
    for t in tree[0]:
        if t.label() == 'NP':
            for s in t.subtrees():
                for n in s.subtrees():
                    if n.label().startswith("NN"):
                        subject = n[0]
                        #subject.append(n[0])
        if t.label() == 'VP':
            for p in t.subtrees():
                for m in p.subtrees():
                    if m.label().startswith("VB"):
                        predicate = m[0]
                        #predicate.append(m[0])
        if t.label() == 'VP':
            for k in t.subtrees(lambda n: n.label() in ['NP', 'PP', 'ADJP']):
                if k.label() in ['NP', 'PP']:
                    for c in k.subtrees(lambda c: c.label().startswith('NN')):
                        obj = c[0]
                        #obj.append(c[0])
                else:
                    for c in k.subtrees(lambda c: c.label().startswith('JJ')):
                        obj = c[0]
                        #obj.append(c[0])
    #if using list, need to make unique
    #subject = list(set(subject))
    #predicate = list(set(predicate))
    #obj = list(set(obj))
    return [subject, predicate, obj]

In [3]:
#parse first sentence in sentence pair
sentence1_parsed = []
for s in sentence1:
    parsed = spo(s)
    sentence1_parsed.append(parsed)
print(sentence1_parsed)
sentence1_parsed_test = []
for s in sentence1_test:
    parsed = spo(s)
    sentence1_parsed_test.append(parsed)

[['Amrozi', 'distorting', 'evidence'], ['Yucaipa', 'selling', 'Safeway'], ['', 'added', ''], ['shares', 'set', 'high'], ['stock', 'close', 'Exchange'], ['year', 'dropped', 'period'], ['Nasdaq', 'closing', 'Friday'], ['DVD-CCA', 'appealed', 'Court'], ['', 'compared', 'period'], ['market', 'rose', 'percent'], ['court', 'won', 'March'], ['index', 'increased', 'percent'], ['', 'improve', 'year'], ['', 'ask', 'asylum'], ['unit', 'carrying', 'passengers'], ['Rudder', 'was', 'Business'], ['chaos', 'logging', 'controls'], ['network', 'lead', 'sufferers'], ['', 'lost', 'jobs'], ['Finder', 'providing', 'access'], ['watches', 'posted', 'islands'], ['Lauderdale', 'held', 'bail'], ['BMI', 'considered', 'obese'], ['Argentina', 'expires', 'August'], ['children', 'approach', 'children'], ['', 'said', 'concourse'], ['SPX', 'fell', 'percent'], ['Duque', 'return', 'Malenchenko'], ['Singapore', 'totaling', 'trade'], ['gains', 'seen', 'exports'], ['', '', ''], ['operations', 'be', 'cents'], ['', 'have', 'B

In [4]:
#parse second sentence in sentence pair
sentence2_parsed = []
for s in sentence2:
    parsed = spo(s)
    sentence2_parsed.append(parsed)
print(sentence2_parsed)
sentence2_parsed_test = []
for s in sentence2_test:
    parsed = spo(s)
    sentence2_parsed_test.append(parsed)

[['Amrozi', 'distorting', 'evidence'], ['Yucaipa', 'sold', 'Safeway'], ['owners', 'offering', 'sale'], ['shares', 'closing', 'high'], ['shares', 'jumped', 'Friday'], ['year', 'dropped', 'period'], ['IXIC', 'rallied', 'percent'], ['CCA', 'appealed', 'Court'], ['Earnings', 'affected', 'period'], ['Xoma', 'were', 'percent'], ['court', 'endorsed', 'House'], ['stocks', 'was', 'points'], ['Sole', 'improve', 'year'], ['', 'told', 'Sun'], ['Heizler', 'heed', 'lights'], ['unit', 'lead', 'entity'], ['chaos', 'logging', 'controls'], ['network', 'lead', 'sufferers'], ['Mankiw', 'said', ''], ['tool', 'puts', 'location'], ['warnings', 'were', 'islands'], ['Zuccarini', 'held', 'Fla'], ['', '', ''], ['sides', 'expires', 'August'], ['invividuals', 'be', 'situations'], ['Tunick', 'said', ''], ['index', 'sinking', 'yesterday'], ['Malenchenko', 'are', 'station'], ['Singapore', 'is', 'volume'], ['exports', 'increased', ''], ['Exchange', 'closed', 'Loral'], ['', 'beat', 'share'], ['', 'stationed', 'troops']

In [5]:
#sentences where SPO could not parse out any of the subject, predicate, or object are meaningless to us
#Since accurate comparisons cannot be made, remove any pairs affected by this
import pandas as pd
df1 = pd.DataFrame(sentence1_parsed, columns = ["S1", "P1", "O1"])
df2 = pd.DataFrame(sentence2_parsed, columns = ["S2", "P2", "O2"])
df3 = pd.DataFrame(labels, columns = ["label"])
combined_df = df1.join(df2)
combined_df = combined_df.join(df3)
cleaned_df = combined_df[(combined_df.S1 != '') & (combined_df.P1 != '') & (combined_df.O1 != '')]
cleaned_df = cleaned_df[(cleaned_df.S2 != '') & (cleaned_df.P2 != '') & (cleaned_df.O2 != '')]
#now split these back out into separate lists; still need to process those via Word2Vec
df1 = cleaned_df.iloc[:,:3]
df2 = cleaned_df.iloc[:,3:6]
df3 = cleaned_df.iloc[:,6:]
sentence1_cleaned = df1.values.tolist()
sentence2_cleaned = df2.values.tolist()
labels = df3.values.tolist()
#now do the same thing for test data
df1 = pd.DataFrame(sentence1_parsed_test, columns = ["S1", "P1", "O1"])
df2 = pd.DataFrame(sentence2_parsed_test, columns = ["S2", "P2", "O2"])
df3 = pd.DataFrame(labels_test, columns = ["label"])
combined_df = df1.join(df2)
combined_df = combined_df.join(df3)
cleaned_df = combined_df[(combined_df.S1 != '') & (combined_df.P1 != '') & (combined_df.O1 != '')]
cleaned_df = cleaned_df[(cleaned_df.S2 != '') & (cleaned_df.P2 != '') & (cleaned_df.O2 != '')]
display(cleaned_df)
#now split these back out into separate lists; still need to process those via Word2Vec
df1 = cleaned_df.iloc[:,:3]
df2 = cleaned_df.iloc[:,3:6]
df3 = cleaned_df.iloc[:,6:]
sentence1_cleaned_test = df1.values.tolist()
sentence2_cleaned_test = df2.values.tolist()
labels_test = df3.values.tolist()

,S1,P1,O1,S2,P2,O2,label
1,sales,expected,backlash,Co.,prompted,backlash,1
3,storm,hit,Monday,storm,hits,coast,0
6,Quaife,remained,operation,Quaife,was,unprecedented,0
8,aide,allied,Thursday,aide,allied,Thursday,1
9,SPX,was,percent,IXIC,was,percent,0
...,...,...,...,...,...,...,...
1715,Yankees,took,pick,Yankees,selected,pick,1
1718,Crews,dump,rain,Crews,use,travel,0
1719,directors,completed,Nvidia,acquisition,close,quarter,1
1722,Hamilton,remained,attack,morning,talked,attack,0


In [6]:
#Word2Vec conversion. The rationale for using it is to match dimensions and procedure with the paper.
#In actuality, we only ever have a single word for a subject, predicate, or object.
import os
import numpy as np
RANDOM_SEED = 23432098
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

import gensim
from gensim.models import Word2Vec
w2v1 = Word2Vec(sentence1_cleaned, vector_size=50, workers=1, min_count=1)
w2v2 = Word2Vec(sentence2_cleaned, vector_size=50, workers=1, min_count=1)
w2v1_test = Word2Vec(sentence1_cleaned_test, vector_size=50, workers=1, min_count=1)
w2v2_test = Word2Vec(sentence2_cleaned_test, vector_size=50, workers=1, min_count=1)
#so I'm not entirely sure what to do here. I think I will have 2 separate Word2Vec models.
#then to get the sentences_final, pull out the .wv for each word in the sentence and transpose it
sentence1_final = []
sentence2_final = []
sentence1_final_test = []
sentence2_final_test = []
for s in sentence1_cleaned:
    words = []
    for w in s:
        mat = w2v1.wv[w]
        words.append(mat.transpose())
    sentence1_final.append(words)
for s in sentence2_cleaned:
    words = []
    for w in s:
        mat = w2v2.wv[w]
        words.append(mat.transpose())
    sentence2_final.append(words)
for s in sentence1_cleaned_test:
    words = []
    for w in s:
        mat = w2v1_test.wv[w]
        words.append(mat.transpose())
    sentence1_final_test.append(words)
for s in sentence2_cleaned_test:
    words = []
    for w in s:
        mat = w2v2_test.wv[w]
        words.append(mat.transpose())
    sentence2_final_test.append(words)

In [7]:
#define the CNN model
#I am using MaxPool2d as opposed to k-max pooling as we know the sentences should always be the same size
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(50, 17, kernel_size=(3,3), padding=1) #try output channels 17 instead of 50
        self.conv2 = nn.Conv2d(17, 6, kernel_size=(3,3), padding=1) #was originally 50 in 1 out
        self.pool1 = nn.MaxPool2d(3)
        self.pool2 = nn.MaxPool2d(1) #errors if set to 3

    def forward(self, x):
        print(x.shape)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.pool2(x)
        return x

In [8]:
#define the data loaders
#to do this, construct the training data by binding together final sentence1 and sentence2 with their target score
df1 = pd.DataFrame(sentence1_final, columns = ["S1", "P1", "O1"])
df2 = pd.DataFrame(sentence2_final, columns = ["S2", "P2", "O2"])
df3 = pd.DataFrame(labels, columns = ["label"])
combined_df = df1.join(df2)
combined_df = combined_df.join(df3)
train_data = combined_df.values.tolist()
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
#repeat for test data
df1 = pd.DataFrame(sentence1_final_test, columns = ["S1", "P1", "O1"])
df2 = pd.DataFrame(sentence2_final_test, columns = ["S2", "P2", "O2"])
df3 = pd.DataFrame(labels_test, columns = ["label"])
combined_df = df1.join(df2)
combined_df = combined_df.join(df3)
test_data = combined_df.values.tolist()
val_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

In [9]:
#now prepare for training
criterion = nn.MSELoss()
model = SimpleCNN()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
n_epochs = 1
from scipy.spatial.distance import cityblock
import math
import random
from tqdm import tqdm
def train_model(model, train_dataloader, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim
    model.train() # prep model for training
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        for s1s, s1p, s1o, s2s, s2p, s2o, target in tqdm(train_dataloader):
            #first, process s1 and s2 through the model
            #ensure the batch size is accurate
            batch = s1s.shape[0]
            s1 = np.concatenate([s1s,s1p,s1o])
            s1 = np.reshape(s1, (batch,50,3,1))
            s1 = torch.from_numpy(s1)
            s1_processed = model(s1)
            s2 = np.concatenate([s2s,s2p,s2o])
            s2 = np.reshape(s2,(batch,50,3,1))
            s2 = torch.from_numpy(s2)
            s2_processed = model(s2)
            #need to detach to perform manhattan distance calculation
            s1_detached = s1_processed.detach()
            print(s1_detached)
            s2_detached = s2_processed.detach()
            y_hats = torch.empty(target.shape[0])
            for i in range(target.shape[0]):
                s1_detached_i = torch.squeeze(s1_detached[i,:,0,:])
                s2_detached_i = torch.squeeze(s2_detached[i,:,0,:])
                #s1_detached_i = torch.unsqueeze(s1_detached_i, 0)
                #s2_detached_i = torch.unsqueeze(s2_detached_i, 0)
                #now calculate manhattan distance
                print(s1_detached_i.shape)
                manhattan = cityblock(s1_detached_i, s2_detached_i)
                y_hat = math.e ** (-manhattan)
                #normalize y_hat score to 0 or 1 for MSRP data
                if y_hat >= 0.5:
                    y_hat = 1
                else:
                    y_hat = 0
                y_hats[i] = y_hat
            y_hats = y_hats.requires_grad_()
            target = target.float()
            loss = criterion(y_hats,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            curr_epoch_loss.append(loss.cpu().data.numpy())
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")
    return model
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
model = train_model(model, train_loader)

  0%|          | 0/33 [00:00<?, ?it/s]C:\Users\rbrow\anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py:149: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\utils\tensor_numpy.cpp:178.)
  return default_collate([torch.as_tensor(b) for b in batch])
  6%|▌         | 2/33 [00:00<00:01, 18.87it/s]

torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
tensor([[[[0.0423],
          [0.0459],
          [0.0474]],

         [[0.0688],
          [0.0709],
          [0.0775]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]]],


        [[[0.0423],
          [0.0463],
          [0.0472]],

         [[0.0671],
          [0.0715],
          [0.0784]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0023],
          [0.0000],
          [0.0000]]],


        [[[0.0432],
          [0.0447],
          [0.0481]],

         [[0.0672],
          [0.0717],
          [0.0791]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0

 39%|███▉      | 13/33 [00:00<00:00, 43.51it/s]

torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
tensor([[[[0.0425],
          [0.0454],
          [0.0475]],

         [[0.0681],
          [0.0707],
          [0.0768]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0023],
          [0.0000],
          [0.0000]]],


        [[[0.0423],
          [0.0463],
          [0.0469]],

         [[0.0675],
          [0.0706],
          [0.0778]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0.0012],
          [0.0000],
          [0.0000]]],


        [[[0.0425],
          [0.0458],
          [0.0482]],

         [[0.0673],
          [0.0710],
          [0.0796]],

         [[0.0000],
          [0.0000],
          [0.0000]],

         [[0

 73%|███████▎  | 24/33 [00:00<00:00, 46.11it/s]

torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Si

100%|██████████| 33/33 [00:00<00:00, 45.07it/s]

torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Size([6])
torch.Si

In [11]:
#Evaluate the model on the test data
def eval_model(model, dataloader):
    model.eval()
    Y_pred = []
    Y_true = []
    for s1s, s1p, s1o, s2s, s2p, s2o, target in dataloader:
        
        batch = s1s.shape[0]
        s1 = np.concatenate([s1s,s1p,s1o])
        s1 = np.reshape(s1, (batch,50,3,1))
        s1 = torch.from_numpy(s1)
        s1_processed = model(s1)
        s2 = np.concatenate([s2s,s2p,s2o])
        s2 = np.reshape(s2,(batch,50,3,1))
        s2 = torch.from_numpy(s2)
        s2_processed = model(s2)
        s1_detached = s1_processed.detach()
        s2_detached = s2_processed.detach()
        y_hats = torch.empty(target.shape[0])
        for i in range(target.shape[0]):
            s1_detached_i = torch.squeeze(s1_detached[i,:,0,:])
            s2_detached_i = torch.squeeze(s2_detached[i,:,0,:])
            #s1_detached_i = torch.unsqueeze(s1_detached_i, 0)
            #s2_detached_i = torch.unsqueeze(s2_detached_i, 0)
            #now calculate manhattan distance
            manhattan = cityblock(s1_detached_i, s2_detached_i)
            y_hat = math.e ** (-manhattan)
            #normalize y_hat score to 0 or 1 for MSRP data
            if y_hat >= 0.5:
                y_hat = 1
            else:
                y_hat = 0
            y_hats[i] = y_hat
        Y_pred.append(y_hats)
        Y_true.append(target)
    Y_pred = np.concatenate(Y_pred, axis=0)
    Y_true = np.concatenate(Y_true, axis=0)
    return Y_pred, Y_true
#print metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

y_pred, y_true = eval_model(model, val_loader)
acc = accuracy_score(y_true, y_pred)
prec, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(acc)
print(prec)
print(recall)
print(fscore)

torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([64, 50, 3, 1])
torch.Size([61, 50, 3, 1])
torch.Size([61, 50, 3, 1])
0.6687565308254964
0.6687565308254964
1.0
0.8015028177833438


In [ ]:
print("Total running time = {:.2f} seconds".format(time.time() - _START_RUNTIME))
print(tracemalloc.get_traced_memory())
tracemalloc.stop()